## Building flask api

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '../../')

In [3]:
from src.serve import *

In [4]:
np.set_printoptions(edgeitems=10, threshold=40, linewidth=200)

In [5]:
data_dir = 'data/midi/v17/'

file_path = Path('../../')
path = file_path/data_dir/'sf4/midi_encode'
out_path = file_path/'data/generated/'

In [6]:
encoded_path = file_path/'data/assets/v17/seed'

## Learn config

In [7]:
import hashlib
import shutil

In [8]:
def df2records(path):
    df = pd.read_csv(path/'midi_encode.csv')
    df = df.loc[df['numpy'].notna()] # make sure it exists
    df = df.loc[df.source == 'hooktheory_c'] # hooktheory only
    genres = 'Pop|Electronic|House|Disney|Classical|Soundtrack|Holiday'
    df = df[df.genres.str.contains(genres, na=False, regex=True)]
    df = df.reindex(index=df.index[::-1]) # A's first
    df = df.where((pd.notnull(df)), None) # nan values break json
    return df.to_dict('records')

In [9]:
# df = pd.read_csv('../../data/midi/v17/sf4/s2s_encode/s2s_encode.csv')
# df = df.loc[df['numpy'].notna()]
# df = df.loc[df.source == 'hooktheory_c']
# df = df[df.genres.str.contains('Pop|Electronic|House|Disney|Classical|Soundtrack|Holiday', na=False, regex=True)]

# from collections import Counter

# c = Counter([g for gs in df.genres.values if isinstance(gs, str) for g in gs.split(',') ]); c

In [10]:
def valid_song(p, min_ts=12, min_var=4):
    if p.shape[0] < min_ts: return False
    if len(set(p[:,0])) < min_var: return False
    return True

In [11]:
def format_meta(s):
    title = s['title'].title().replace('-', ' ')
    artist = s['artist'].title().replace('-', ' ')
    sid = s['md5']
    display = f'{title} - {artist}'
    
    # filter
    m = np.load(file_path/data_dir/s['numpy'], allow_pickle=True)
    if not (valid_song(m, min_ts=50, min_var=7)): 
        print('Invalid song:', s['numpy'])
        return None
    if s['ht_bpm'] < 60: 
        print('Song too slow:', s['numpy'])
        return None
    
    json_data = {
        'title': title,
        'artist': artist,
        'bpm': s['ht_bpm'],
        'section': s['section'],
        'genres': s['genres'],
        'display': display,
        'sid': sid
    }
    source_file = file_path/data_dir/s['midi']
#     key_file = str(source_file).replace('cmajor', 'original')
    key_file = source_file
    to_file = encoded_path/f'{sid[::-1]}.mid'
    to_json = encoded_path/f'{sid[::-1]}.json'
    if not to_file.exists():
        to_file.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(str(key_file), str(to_file))
        with open(to_json, 'w') as jp:
            json.dump(json_data, jp)
    
    return json_data

In [16]:
def build_db(path):
    recordlist = df2records(path)
    htlist = [format_meta(s) for s in recordlist]
    htlist = [s for s in htlist if s is not None]
    json_path = file_path/'data/assets/v17/json/htlist.json'
    json_path.parent.mkdir(parents=True, exist_ok=True)
    with open(json_path, 'w') as fp:
        json.dump(htlist, fp, separators=(',', ':'))
    return htlist

In [17]:
htlist = build_db(path)

Invalid song: sf4/midi_encode/hooktheory/pianoroll/a/above-and-beyond/you-got-to-go/bridge_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/a/above-and-beyond/good-for-me---ambient-mix--/intro_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/a/a-g-cook/superstar/pre-chorus_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/a/au5/inside-ft-danyka-nadeau/chorus_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/a/adam-neely/have-yourself-a-merry-little-christmas/intro_key_cmajor.npy
Song too slow: sf4/midi_encode/hooktheory/pianoroll/a/alan-silvestri/the-cosmos-is-yours---cosmos-a-spacetime-odyssey/instrumental_key_cmajor.npy
Song too slow: sf4/midi_encode/hooktheory/pianoroll/a/arnold-schoenberg/verklarte-nacht---sehr-breit-und-langsam/chorus_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/a/afta-1/flyerthanshe/instrumental_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/a/afta-1/quest/in

Invalid song: sf4/midi_encode/hooktheory/pianoroll/t/tori-kelly/confetti/pre-chorus_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/t/the-naked-and-famous/rolling-waves/intro_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/t/the-naked-and-famous/rolling-waves/chorus_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/t/togakushi-touko/midday-star/pre-chorus_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/t/togakushi-touko/midday-star/verse_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/t/timeflies/all-the-way---milkman-remix/intro_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/t/tommy-heavenly6/papermoon--soul-eater-op-2/outro_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/t/the-rembrants/ill-be-there-for-you/intro_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/t/totally-enormous-extinct-dinosaurs-teed-t-e-e-d/household-goods/chorus_key_cmajor.

Invalid song: sf4/midi_encode/hooktheory/pianoroll/s/stephen/crossfire/verse_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/s/sasakureuk/chronicleseven/intro_key_cmajor.npy
Song too slow: sf4/midi_encode/hooktheory/pianoroll/s/sarah-fuller-flower-adams/nearer-my-god-to-thee/instrumental_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/s/steerner--tjernberg--beerg--haal/names/chorus_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/d/disasterpeace/jay/intro_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/d/disney/be-our-guest---beauty-and-the-beast/solo_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/d/deadmau5/faxing-berlin/chorus_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/d/deadmau5/raise-your-weapon---madeon-remix/chorus_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/d/deadmau5/ghosts-n-stuff/intro_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pi

Invalid song: sf4/midi_encode/hooktheory/pianoroll/j/jack-u/take-u-there-feat-kiesza/intro_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/j/jaytech/multiverse---audien-remix/bridge_key_cmajor.npy
Song too slow: sf4/midi_encode/hooktheory/pianoroll/j/johann-sebastian-bach/bwv-4---sinfonia---christ-lag-in-todesbanden/instrumental_key_cmajor.npy
Song too slow: sf4/midi_encode/hooktheory/pianoroll/j/johann-sebastian-bach/air-on-the-g-string/verse_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/j/josh-guru-project/infinity-2008/instrumental_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/j/jewel/intuition/intro_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/j/james-blunt/1973/bridge_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/j/jacksfilms/valentines-day-is-a-lie/bridge_key_cmajor.npy
Invalid song: sf4/midi_encode/hooktheory/pianoroll/j/jacksfilms/valentines-day-is-a-lie/verse_key_cmajor.npy
Inval

In [18]:
len(htlist)

5918

In [19]:
htlist

[{'title': 'Famous',
  'artist': 'Archie',
  'bpm': 128.0,
  'section': 'chorus',
  'genres': 'Electronic',
  'display': 'Famous - Archie',
  'sid': '67a12c729c20a3bc49e22e1da46d472e'},
 {'title': 'You Learn',
  'artist': 'Alanis Morissette',
  'bpm': 84.0,
  'section': 'chorus',
  'genres': 'Alternative,Pop,Rock',
  'display': 'You Learn - Alanis Morissette',
  'sid': 'f9b7ea35a536655b820cc667f541d3c9'},
 {'title': 'Ironic',
  'artist': 'Alanis Morissette',
  'bpm': 86.0,
  'section': 'chorus',
  'genres': 'Pop,Rock',
  'display': 'Ironic - Alanis Morissette',
  'sid': '369373d75ebc0a0afde1e561af9fe4b6'},
 {'title': 'Lovelight   Bad Apple',
  'artist': 'Alstroemeria Records',
  'bpm': 138.0,
  'section': 'bridge',
  'genres': 'J-Pop',
  'display': 'Lovelight   Bad Apple - Alstroemeria Records',
  'sid': 'a8c214961de4d9b7c8832738a056b174'},
 {'title': 'Lovelight   Bad Apple',
  'artist': 'Alstroemeria Records',
  'bpm': 138.0,
  'section': 'verse',
  'genres': 'J-Pop',
  'display': 'Lo